![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import glob
import os
import random
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
from datetime import datetime
from rapidfuzz import process, fuzz

In [2]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [3]:
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "AMS_Gallery_2024_25","*.csv"))
filepath_list.append(glob.glob(os.path.join(os.getcwd(), "data", "archive", "AMS","AMS_Gallery_Data","Gallery_all_years_test_new.csv"))[0])
filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/AMS_Gallery_2024_25/Gallery_2024S_Prep_Recipe_Detail.csv',
 '/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/AMS_Gallery_2024_25/Gallery_2024S_Menu_Product_Detail.csv',
 '/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/archive/AMS/AMS_Gallery_Data/Gallery_all_years_test_new.csv']

In [4]:
Preps = pd.read_csv(filepath_list[0], skiprows = 1, header=None)
Preps.columns = ["Empty","category_1","category_2", "item_descrip","batch_yield", "total_cost", "Total Cost", "Shelf_Life", "Shelf Lifr", "recipe_uom","recipe_cost", "recipe_uom_2","recipe_cost_2","line_qty","ingre_cost","item_descrip.1","uom"]
Preps = Preps.drop(columns=["Total Cost","Shelf_Life","Shelf Lifr", "category_1", "Empty","recipe_uom","recipe_cost","recipe_uom_2","recipe_cost_2"])
Preps

,category_2,item_descrip,batch_yield,total_cost,line_qty,ingre_cost,item_descrip.1,uom
0,Beverages - Juice,Raspberry Puree,Batch Yield: 2.000 L,$14.0160,1.00,$14.0160,Raspberries Frozen IQF,Kg
1,Beverages - Juice,Raspberry Puree,Batch Yield: 2.000 L,$14.0160,1.50,$0.0000,Water - Tap,L
2,Beverages - Juice,Strawberry Puree,Batch Yield: 2.000 L,$5.5242,1.00,$5.5242,Strawberries IQF FRZ,Kg
3,Beverages - Juice,Strawberry Puree,Batch Yield: 2.000 L,$5.5242,1.50,$0.0000,Water - Tap,L
4,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0214,CUP PORT PAPER WHT 2Z,ea
...,...,...,...,...,...,...,...,...
1239,Alcohol - White Wine,Sangria White Tab 2023,Batch Yield: 16.000 L,$109.2348,200.00,$4.7647,Bacardi Light Rum 750mL,ml
1240,Alcohol - White Wine,Sangria White Tab 2023,Batch Yield: 16.000 L,$109.2348,200.00,$5.0881,Meagher's Triple Sec,ml
1241,Alcohol - White Wine,Sangria White Tab 2023,Batch Yield: 16.000 L,$109.2348,3.00,$6.0833,JUICE orange 100% tetra,L
1242,Alcohol - White Wine,Sangria White Tab 2023,Batch Yield: 16.000 L,$109.2348,375.00,$9.6546,Malibu 1.14L,ml


In [5]:
Items = pd.read_csv(filepath_list[1], skiprows = 1, header=None)
Items.columns = ["Empty","category_1","category_2","Warning","Warning.1", "item_descrip","Empty.1", "drop","drop.1","drop.2","drop.3","drop.4","drop.5","PLU","drop.6","uom","item_descrip.1"]
Items = Items.drop(columns=["category_1","category_2","Warning","Warning.1","Empty","Empty.1","drop","drop.1","drop.2","drop.3","drop.4","drop.5","drop.6"])
Items

,item_descrip,PLU,uom,item_descrip.1
0,Lone Tree Cider,1.0,can,Lone Tree Cider Can
1,160z Twin Sails IPA 2023,16.0,oz (fl),Twin Sails Juice Plus Citra
2,16oz 33 Acers French B 2023,16.0,oz (fl),33 Acres French Blanche
3,16oz Beer League lager 2023,16.0,oz (fl),Central City Lager
4,16oz Fuggles Plum Sour 2023,16.0,oz (fl),Fuggles Plum Sour
...,...,...,...,...
891,6oz Antares White 2023,6.0,oz (fl),Antares Sauvignon Blanc
892,Bottle Antares Red 2023,1.0,750ml,Antares Carmenere
893,Bottle Antares White 2023,1.0,750ml,Antares Sauvignon Blanc
894,HH House wine Red 2024,6.0,oz (fl),Antares Carmenere


In [6]:
Products = Items[Items["item_descrip"].str.contains("2024S")]
Products = Products.reset_index(drop=True)
Products

,item_descrip,PLU,uom,item_descrip.1
0,2024S Beets Salad,1.0,ea,2023 Beets Salad Prep
1,2024S Beets Salad,1.0,ea,Misc Extras
2,2024S Blackend Chick Carbonara,1.0,ea,2023 Blackened Carbonara Prep
3,2024S Blackend Chick Carbonara,2.0,Tbsp,2023 Herb oil
4,2024S Blackend Chick Carbonara,1.0,ea,Misc Extras
...,...,...,...,...
136,2024S Wing Honey Garlic,20.0,g,BUTTER REG SALTED
137,2024S Wing Honey Garlic,30.0,g,HONEY LIQUID SQUEEZE CAN
138,2024S Wing Honey Garlic,1.0,g,SALT KOSHER COARSE
139,2024S Wing Honey Garlic,0.1,bunch,PARSLEY


In [7]:
# Get unique values from the 'item_descrip.2' column in Products
prods = Products["item_descrip.1"].unique()

# Get unique values from the 'item_descrip.1' column in Preps
preps_unique = Preps["item_descrip.1"].unique()

# Filter Preps DataFrame to keep only rows where 'item_descrip' is in 'prods' or 'preps_unique'
Preps = Preps[Preps["item_descrip"].isin(prods) | Preps["item_descrip"].isin(preps_unique)]

# Reset index of Preps DataFrame
Preps = Preps.reset_index(drop=True)
Preps

,category_2,item_descrip,batch_yield,total_cost,line_qty,ingre_cost,item_descrip.1,uom
0,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0214,CUP PORT PAPER WHT 2Z,ea
1,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.1447,LID Foil Board 6x8.5 SO,ea
2,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0169,Napkin Bev 2ply White,ea
3,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0471,CUTLERY FORK WOOD bulk,ea
4,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0471,CUTLERY SPOON WOOD soup,ea
...,...,...,...,...,...,...,...,...
715,Bar Supply,Simple Syrup 2023,Batch Yield: 700.000 ml,$1.1568,440.00,$1.1568,SUGAR GRANULATED FINE,g
716,Misc,Misc Extras,Batch Yield: 1.000 ea,$0.3051,1.00,$0.1648,ECO 9x6 Kraft Hinged Container,ea
717,Misc,Misc Extras,Batch Yield: 1.000 ea,$0.3051,1.00,$0.0480,CUTLERY FORK WOODEN ECO,ea
718,Misc,Misc Extras,Batch Yield: 1.000 ea,$0.3051,1.00,$0.0463,CUTLERY SPOON WOODEN ECO,ea


In [8]:
old_items = pd.read_csv(filepath_list[-1])
old_items = old_items.drop(columns=['item_num.2', 'item_num.1'])
old_items

,item_num,item_descrip,pak_physical_yield,pak_uom,pak_uom.1,pak_factored_cost,line_item_num,line_qty,items_comments,item_descrip.1,inv_flag,uom
0,17284,2022 goose & Watermelon bull,1.0,PORT,ea,0.559100,1971,0.25,NaN,LIMES,N,ea
1,17284,2022 goose & Watermelon bull,1.0,PORT,fl oz,1.553600,8228,1.00,NaN,Grey Goose 1.14L,N,fl oz
2,17284,2022 goose & Watermelon bull,1.0,PORT,can,1.920800,15803,1.00,NaN,Red Bull Watermelon,N,can
3,18292,Vegan Caesar wrap 2022,1.0,ea,HEAD,2.466700,5505,0.25,NaN,Lettuce - Romaine,N,HEAD
4,18292,Vegan Caesar wrap 2022,1.0,ea,ml,0.189270,8667,2.00,NaN,ITEM GARLIC MAYO,N,fl oz
...,...,...,...,...,...,...,...,...,...,...,...,...
2153,18049,Yellow Curry Prep Gall.2023,1.0,PORT,g,0.005200,2262,1.00,NaN,SESAME SEEDS,Y,g
2154,18049,Yellow Curry Prep Gall.2023,1.0,PORT,oz,0.003968,14434,80.00,NaN,Potato yellow Med 5lb bag,Y,g
2155,18049,Yellow Curry Prep Gall.2023,1.0,PORT,PORT,0.200000,15477,1.00,NaN,2022 Jasmin rice,Y,PORT
2156,18049,Yellow Curry Prep Gall.2023,1.0,PORT,g,0.005300,15637,20.00,NaN,Whole Green Beans IQF,Y,g


In [9]:
used_ids = []
def assign_ids(df):
    # Preprocessing function
    def preprocess(description):
        description = description.lower()
        description = re.sub(r'\s+', ' ', description).strip()
        description = re.sub(r'[^\w\s]', '', description)
        return description

    # Apply preprocessing
    old_items['Normalized_Description'] = old_items['item_descrip'].apply(preprocess)
    df['Normalized_Description'] = df['item_descrip'].apply(preprocess)

    # Create a mapping from normalized description to item_num
    desc_to_item_num = {row['Normalized_Description']: row['item_num'] for idx, row in old_items.iterrows()}

    # Exact match
    df['item_num'] = df['Normalized_Description'].apply(lambda x: desc_to_item_num.get(x))

    # Fuzzy matching function
    def get_all_matches(description, choices, threshold=80):
        matches = process.extract(description, choices, scorer=fuzz.token_sort_ratio)
        return [match for match in matches if match[1] >= threshold]

    # Apply fuzzy matching where exact match failed
    df['Potential_Matches'] = df['Normalized_Description'].apply(lambda x: get_all_matches(x, desc_to_item_num.keys()))

    # Assign item_num directly if there is exactly one potential match
    def assign_item_num_or_matches(row):
        potential_matches = row['Potential_Matches']
        if potential_matches and len(potential_matches) == 1:
            # Assign the item_num of the single potential match
            return desc_to_item_num.get(potential_matches[0][0])
        else:
            # Keep as None if no match or multiple matches
            return None

    # Apply the function to assign item_num
    df['item_num'] = df.apply(lambda row: assign_item_num_or_matches(row), axis=1)

    # Add the best potential match item_num to Items
    def add_best_potential_match_item_num(row):
        potential_matches = row['Potential_Matches']
        if len(potential_matches) > 1:
            best_match = max(potential_matches, key=lambda x: x[1])
            return desc_to_item_num.get(best_match[0])
        return row['item_num']

    # Update Items to include best potential match item_num if exact match fails
    df['item_num'] = df.apply(lambda row: add_best_potential_match_item_num(row), axis=1)

    # Function to generate unique random item_num
    def generate_unique_random_item_num(existing_nums, start=1000):
        random.seed(42) #DO NOT CHANGE
        while True:
            num = random.randint(start, start + 10000)
            if num not in existing_nums and num not in used_ids:
                used_ids.append(num)
                return num

    # Get the set of used item_nums
    used_item_nums = set(old_items['item_num']).union(set(df['item_num'].dropna()))

    # Assign random item_num for rows without one, ensuring same item_descrip gets same item_num
    unassigned = df[df['item_num'].isna()]
    unique_descriptions = unassigned['Normalized_Description'].unique()

    for desc in unique_descriptions:
        random_num = generate_unique_random_item_num(used_item_nums)
        used_item_nums.add(random_num)
        df.loc[df['Normalized_Description'] == desc, 'item_num'] = random_num

    df["item_num"] = df["item_num"].astype(int)
    df.to_csv("df.csv")
    # Optionally, remove the Potential_Matches column if no longer needed
    df.drop(columns=['Potential_Matches',"Normalized_Description"], inplace=True)
    # Print the result
    return df


In [10]:
Preps = assign_ids(Preps)
Products = assign_ids(Products)

In [11]:
#Ensure 'line_item_num' column exists in Items
Preps['line_item_num'] = 0

# Create a dictionary for quick lookup from old_items
old_items_dict = old_items.set_index('item_descrip.1')['line_item_num'].to_dict()
old_ids = list(set(old_items["line_item_num"].unique()))
ID = old_ids[0]

# Iterate through Items and update line_item_num
for index, row in Preps.iterrows():
    description = row['item_descrip.1']
    if description in old_items_dict:
        Preps.at[index, 'line_item_num'] = old_items_dict[description]
    else:
        while ID in old_ids:
            ID += 1
        old_items_dict[description] = ID
        old_ids.append(ID)
        Preps.at[index, "line_item_num"] = ID

# Output the updated DataFrame
display(Preps)

,category_2,item_descrip,batch_yield,total_cost,line_qty,ingre_cost,item_descrip.1,uom,item_num,line_item_num
0,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0214,CUP PORT PAPER WHT 2Z,ea,17297,4099
1,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.1447,LID Foil Board 6x8.5 SO,ea,17297,4100
2,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0169,Napkin Bev 2ply White,ea,17297,4101
3,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0471,CUTLERY FORK WOOD bulk,ea,17297,4102
4,Kitchen Supplies,To Go Cutlery 2023,Batch Yield: 1.000 ea,$0.6308,1.00,$0.0471,CUTLERY SPOON WOOD soup,ea,17297,4103
...,...,...,...,...,...,...,...,...,...,...
715,Bar Supply,Simple Syrup 2023,Batch Yield: 700.000 ml,$1.1568,440.00,$1.1568,SUGAR GRANULATED FINE,g,16844,2367
716,Misc,Misc Extras,Batch Yield: 1.000 ea,$0.3051,1.00,$0.1648,ECO 9x6 Kraft Hinged Container,ea,4814,4239
717,Misc,Misc Extras,Batch Yield: 1.000 ea,$0.3051,1.00,$0.0480,CUTLERY FORK WOODEN ECO,ea,4814,9601
718,Misc,Misc Extras,Batch Yield: 1.000 ea,$0.3051,1.00,$0.0463,CUTLERY SPOON WOODEN ECO,ea,4814,9599


In [12]:
def extract_yield_and_uom(batch_yield):
    match = re.search(r'Batch\s*Yield:\s*([\d.]+)\s*(\w+)', batch_yield)
    if match:
        physical_yield = float(match.group(1))
        uom = match.group(2)
        return physical_yield, uom
    else:
        print(f"Error parsing batch_yield '{batch_yield}': pattern not found")
        return None, None

# Apply the function and create new columns
Preps['pak_physical_yield'], Preps['pak_uom'] = zip(*Preps['batch_yield'].apply(extract_yield_and_uom))
Preps = Preps.drop(columns=["batch_yield"])
Preps

,category_2,item_descrip,total_cost,line_qty,ingre_cost,item_descrip.1,uom,item_num,line_item_num,pak_physical_yield,pak_uom
0,Kitchen Supplies,To Go Cutlery 2023,$0.6308,1.00,$0.0214,CUP PORT PAPER WHT 2Z,ea,17297,4099,1.0,ea
1,Kitchen Supplies,To Go Cutlery 2023,$0.6308,1.00,$0.1447,LID Foil Board 6x8.5 SO,ea,17297,4100,1.0,ea
2,Kitchen Supplies,To Go Cutlery 2023,$0.6308,1.00,$0.0169,Napkin Bev 2ply White,ea,17297,4101,1.0,ea
3,Kitchen Supplies,To Go Cutlery 2023,$0.6308,1.00,$0.0471,CUTLERY FORK WOOD bulk,ea,17297,4102,1.0,ea
4,Kitchen Supplies,To Go Cutlery 2023,$0.6308,1.00,$0.0471,CUTLERY SPOON WOOD soup,ea,17297,4103,1.0,ea
...,...,...,...,...,...,...,...,...,...,...,...
715,Bar Supply,Simple Syrup 2023,$1.1568,440.00,$1.1568,SUGAR GRANULATED FINE,g,16844,2367,700.0,ml
716,Misc,Misc Extras,$0.3051,1.00,$0.1648,ECO 9x6 Kraft Hinged Container,ea,4814,4239,1.0,ea
717,Misc,Misc Extras,$0.3051,1.00,$0.0480,CUTLERY FORK WOODEN ECO,ea,4814,9601,1.0,ea
718,Misc,Misc Extras,$0.3051,1.00,$0.0463,CUTLERY SPOON WOODEN ECO,ea,4814,9599,1.0,ea


In [13]:
Products["inv_flag"] = "Y"
Preps["inv_flag"] = "Y"

In [14]:
Preps.to_csv("Preps.csv", index=False)
Products.to_csv("Products.csv", index=False)

### Import Items List

In [15]:
Preps = Preps[(Preps["item_descrip"] != "To Go Cutlery 2023") & (Preps["item_descrip"] != "Misc Extras")]
Preps.reset_index(drop=True, inplace=True)
Preps

,category_2,item_descrip,total_cost,line_qty,ingre_cost,item_descrip.1,uom,item_num,line_item_num,pak_physical_yield,pak_uom,inv_flag
0,Commissary,Pro - Kansas City BBQ,$5.2557,20.00,$0.1608,JALAPENO PEPPER - FRESH,g,6068,1927,1.25,L,Y
1,Commissary,Pro - Kansas City BBQ,$5.2557,660.00,$2.1109,KETCHUP VOL PAK,ml,6068,1958,1.25,L,Y
2,Commissary,Pro - Kansas City BBQ,$5.2557,22.50,$0.7488,MUSTARD DRY,g,6068,2025,1.25,L,Y
3,Commissary,Pro - Kansas City BBQ,$5.2557,15.00,$0.1752,SPICE CAYENNE SHAKER,ml,6068,2323,1.25,L,Y
4,Commissary,Pro - Kansas City BBQ,$5.2557,20.00,$0.3395,SPICE CHILI POWDER,g,6068,2324,1.25,L,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
705,Prepared Frz,Pro - Marinara Sauce,$19.9424,60.00,$2.1495,BASIL Fresh,g,15368,3005,7.00,L,Y
706,Prepared Frz,Pro - Marinara Sauce,$19.9424,18.00,$0.3147,SPICE CHILI RED PEPPER CRUSHED,g,15368,3804,7.00,L,Y
707,Prepared Frz,Pro - Marinara Sauce,$19.9424,500.00,$2.2223,Onion White Peeled,g,15368,6865,7.00,L,Y
708,Prepared Frz,Pro - Marinara Sauce,$19.9424,1.00,$6.7740,Sawmill Creek - White 16L,L,15368,9117,7.00,L,Y


In [16]:
# Ensure 'item_num' and 'line_item_num' are strings
Preps = Preps.astype({"item_num": str, "line_item_num": str})

# Get unique descriptions from 'item_descrip'
unique_descriptions = Preps["item_descrip"].unique()

# Iterate over rows to update 'item_num' and 'line_item_num'
for idx, row in Preps.iterrows():
    # Prepend 'P-' if not already present in 'item_num'
    if not row["item_num"].startswith("P-"):
        Preps.at[idx, "item_num"] = "P-" + row["item_num"]
    
for idx, row in Preps.iterrows(): 
    # Update 'line_item_num' based on 'item_descrip'
    if row["item_descrip.1"] in unique_descriptions:
        matching_item_num = Preps.loc[Preps["item_descrip"] == row["item_descrip.1"], "item_num"].values[0]
        print(f"Matching item_num: {matching_item_num}")
        Preps.at[idx, "line_item_num"] = matching_item_num
    else:
        # Prepend 'I-' if not already present in 'line_item_num'
        if not row["line_item_num"].startswith("I-"):
            Preps.at[idx, "line_item_num"] = "I-" + row["line_item_num"]

# Optional: Verify the changes
Preps.head(20)


Matching item_num: P-18052
Matching item_num: P-6881
Matching item_num: P-18052
Matching item_num: P-16778
Matching item_num: P-18274
Matching item_num: P-12209
Matching item_num: P-18349
Matching item_num: P-17486
Matching item_num: P-17037
Matching item_num: P-16856
Matching item_num: P-17037
Matching item_num: P-11706
Matching item_num: P-3286
Matching item_num: P-2679
Matching item_num: P-11706
Matching item_num: P-11125
Matching item_num: P-16222
Matching item_num: P-16221
Matching item_num: P-18334
Matching item_num: P-12209
Matching item_num: P-18268
Matching item_num: P-18336
Matching item_num: P-16571
Matching item_num: P-16571
Matching item_num: P-17360
Matching item_num: P-2679
Matching item_num: P-16778
Matching item_num: P-3547
Matching item_num: P-15007
Matching item_num: P-2824
Matching item_num: P-18349
Matching item_num: P-18329
Matching item_num: P-11706
Matching item_num: P-16855
Matching item_num: P-16780
Matching item_num: P-16856
Matching item_num: P-9279
Matching

,category_2,item_descrip,total_cost,line_qty,ingre_cost,item_descrip.1,uom,item_num,line_item_num,pak_physical_yield,pak_uom,inv_flag
0,Commissary,Pro - Kansas City BBQ,$5.2557,20.00,$0.1608,JALAPENO PEPPER - FRESH,g,P-6068,I-1927,1.25,L,Y
1,Commissary,Pro - Kansas City BBQ,$5.2557,660.00,$2.1109,KETCHUP VOL PAK,ml,P-6068,I-1958,1.25,L,Y
2,Commissary,Pro - Kansas City BBQ,$5.2557,22.50,$0.7488,MUSTARD DRY,g,P-6068,I-2025,1.25,L,Y
3,Commissary,Pro - Kansas City BBQ,$5.2557,15.00,$0.1752,SPICE CAYENNE SHAKER,ml,P-6068,I-2323,1.25,L,Y
4,Commissary,Pro - Kansas City BBQ,$5.2557,20.00,$0.3395,SPICE CHILI POWDER,g,P-6068,I-2324,1.25,L,Y
5,Commissary,Pro - Kansas City BBQ,$5.2557,1.00,$0.0299,SPICE CUMIN SEED GROUND BOX,g,P-6068,I-2329,1.25,L,Y
6,Commissary,Pro - Kansas City BBQ,$5.2557,5.00,$0.1167,SPICE GARLIC GRANULATED,g,P-6068,I-4105,1.25,L,Y
7,Commissary,Pro - Kansas City BBQ,$5.2557,10.00,$0.3582,SPICE Pepper Black Grnd,g,P-6068,I-4106,1.25,L,Y
8,Commissary,Pro - Kansas City BBQ,$5.2557,200.00,$0.4892,SUGAR GOLDEN YEL,g,P-6068,I-2366,1.25,L,Y
9,Commissary,Pro - Kansas City BBQ,$5.2557,200.00,$0.0000,Water - Tap,ml,P-6068,I-2640,1.25,L,Y


In [17]:
Preps.dtypes

category_2             object
item_descrip           object
total_cost             object
line_qty               object
ingre_cost             object
item_descrip.1         object
uom                    object
item_num               object
line_item_num          object
pak_physical_yield    float64
pak_uom                object
inv_flag               object
dtype: object

In [18]:
Preps.to_csv("Preps.csv", index=False)

In [19]:
Products = Products[(Products["item_descrip.1"] != "To Go Cutlery 2023") & (Products["item_descrip.1"] != "Misc Extras")]
Products

,item_descrip,PLU,uom,item_descrip.1,item_num,inv_flag
0,2024S Beets Salad,1.0,ea,2023 Beets Salad Prep,2654,Y
2,2024S Blackend Chick Carbonara,1.0,ea,2023 Blackened Carbonara Prep,7227,Y
3,2024S Blackend Chick Carbonara,2.0,Tbsp,2023 Herb oil,7227,Y
5,2024S Chicken Caesar (Wrap),1.0,ea,2023 Chicken Caesar wrap Prep,8990,Y
7,2024S Chicken Pesto Penne,1.0,ea,2023 Pesto Chicken,5554,Y
...,...,...,...,...,...,...
135,2024S Wing Honey Garlic,5.0,g,GARLIC WHOLE PEELED,2084,Y
136,2024S Wing Honey Garlic,20.0,g,BUTTER REG SALTED,2084,Y
137,2024S Wing Honey Garlic,30.0,g,HONEY LIQUID SQUEEZE CAN,2084,Y
138,2024S Wing Honey Garlic,1.0,g,SALT KOSHER COARSE,2084,Y


In [20]:
#Ensure 'line_item_num' column exists in Items
Products.loc[:,'line_item_num'] = ""

# Create a dictionary for quick lookup from old_items
items_dict = Preps.set_index('item_descrip.1')['line_item_num'].to_dict()
items_dict.update(Preps.set_index('item_descrip')['item_num'].to_dict())

ids = list(set(Preps["line_item_num"].unique()))
ids.extend(Preps["item_num"].unique())
ids = [int(str(id).split("-")[1]) for id in ids]
print(ids)
ID = ids[0]

# Iterate through Items and update line_item_num
for index, row in Products.iterrows():
    description = row['item_descrip.1']
    if description in items_dict:
        Products.at[index, 'line_item_num'] = items_dict[description]
    else:
        while ID in ids:
            ID += 1
        print(row["item_descrip.1"], "not in Preps")
        items_dict[description] = "I-" + str(ID)
        ids.append(ID)
        Products.at[index, "line_item_num"] = "I-" + str(ID)

# Output the updated DataFrame
display(Products)

[4194, 1917, 1859, 4118, 5777, 17946, 4148, 16793, 1655, 1588, 4107, 6881, 4226, 5333, 4204, 2068, 2139, 2323, 4189, 13842, 17486, 4119, 1927, 5008, 16778, 4192, 4202, 4129, 1941, 16748, 3615, 4220, 5505, 1750, 6204, 2329, 4108, 9279, 4232, 4160, 14484, 1874, 2037, 2087, 4140, 18268, 15368, 2436, 4141, 18380, 1660, 4205, 1995, 16575, 4153, 2640, 3354, 4124, 4152, 4158, 3286, 16780, 4236, 4139, 3258, 4109, 15200, 4125, 6514, 4128, 7224, 8975, 3804, 2400, 4186, 16144, 2443, 14126, 1610, 4133, 7775, 6925, 4091, 6461, 2347, 5941, 1635, 2324, 14422, 8851, 14977, 18329, 4127, 7645, 2339, 14556, 4184, 4211, 17360, 4105, 4582, 4166, 12132, 4131, 2183, 18052, 7656, 4172, 4135, 17129, 4235, 5557, 2093, 4171, 14532, 4231, 9117, 14790, 18336, 1789, 4208, 3180, 16855, 13736, 5731, 2378, 5575, 15221, 4162, 17977, 4206, 16699, 4014, 1649, 10834, 13691, 5741, 2153, 18349, 1908, 15007, 11125, 14429, 2679, 2102, 4174, 9935, 4195, 4147, 2366, 16856, 11706, 5126, 14419, 7338, 4229, 4190, 2464, 4188, 1892,

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_70459/3281539405.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Products.loc[:,'line_item_num'] = ""


,item_descrip,PLU,uom,item_descrip.1,item_num,inv_flag,line_item_num
0,2024S Beets Salad,1.0,ea,2023 Beets Salad Prep,2654,Y,P-18275
2,2024S Blackend Chick Carbonara,1.0,ea,2023 Blackened Carbonara Prep,7227,Y,P-5506
3,2024S Blackend Chick Carbonara,2.0,Tbsp,2023 Herb oil,7227,Y,P-17013
5,2024S Chicken Caesar (Wrap),1.0,ea,2023 Chicken Caesar wrap Prep,8990,Y,P-14560
7,2024S Chicken Pesto Penne,1.0,ea,2023 Pesto Chicken,5554,Y,P-18295
...,...,...,...,...,...,...,...
135,2024S Wing Honey Garlic,5.0,g,GARLIC WHOLE PEELED,2084,Y,I-1874
136,2024S Wing Honey Garlic,20.0,g,BUTTER REG SALTED,2084,Y,I-2037
137,2024S Wing Honey Garlic,30.0,g,HONEY LIQUID SQUEEZE CAN,2084,Y,I-1917
138,2024S Wing Honey Garlic,1.0,g,SALT KOSHER COARSE,2084,Y,I-4091


In [21]:
Products.to_csv("Products.csv", index=False)

### Extracting all Preps

In [22]:
# Filter the DataFrame to include only rows where 'item_num' starts with 'P-'
Preperations = Preps.copy()

# Select specific columns from the filtered DataFrame
Preperations = Preperations[['item_num', 'item_descrip', 'pak_physical_yield', 'pak_uom', 'inv_flag']]
Preperations.rename(columns={'item_num': 'PrepId', 'item_descrip': 'Description', 'pak_physical_yield': 'PakQty', 'pak_uom': 'PakUOM', 'inv_flag': 'InventoryGroup'}, inplace=True)
Preperations.drop_duplicates(subset=["PrepId"], inplace=True)
Preperations.reset_index(drop=True, inplace=True)
Preperations

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-6068,Pro - Kansas City BBQ,1.25,L,Y
1,P-2824,2023 Alfredo Sauce Gal.,2250.00,ml,Y
2,P-1409,2023 Basmati Prep,2.00,Kg,Y
3,P-17360,2023 Beef Gravy (prep),4.50,L,Y
4,P-18275,2023 Beets Salad Prep,1.00,ea,Y
...,...,...,...,...,...
109,P-4733,Tzatziki,4.00,Kg,Y
110,P-5741,Veggie Stock,6.00,Kg,Y
111,P-2307,Waffles,55.00,ea,Y
112,P-15368,Pro - Marinara Sauce,7.00,L,Y


In [23]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "AMS_data", "Preps_List.csv")
Preperations.to_csv(path, index = False, header = True)

### Extracting all Items

In [24]:
Preps

,category_2,item_descrip,total_cost,line_qty,ingre_cost,item_descrip.1,uom,item_num,line_item_num,pak_physical_yield,pak_uom,inv_flag
0,Commissary,Pro - Kansas City BBQ,$5.2557,20.00,$0.1608,JALAPENO PEPPER - FRESH,g,P-6068,I-1927,1.25,L,Y
1,Commissary,Pro - Kansas City BBQ,$5.2557,660.00,$2.1109,KETCHUP VOL PAK,ml,P-6068,I-1958,1.25,L,Y
2,Commissary,Pro - Kansas City BBQ,$5.2557,22.50,$0.7488,MUSTARD DRY,g,P-6068,I-2025,1.25,L,Y
3,Commissary,Pro - Kansas City BBQ,$5.2557,15.00,$0.1752,SPICE CAYENNE SHAKER,ml,P-6068,I-2323,1.25,L,Y
4,Commissary,Pro - Kansas City BBQ,$5.2557,20.00,$0.3395,SPICE CHILI POWDER,g,P-6068,I-2324,1.25,L,Y
...,...,...,...,...,...,...,...,...,...,...,...,...
705,Prepared Frz,Pro - Marinara Sauce,$19.9424,60.00,$2.1495,BASIL Fresh,g,P-15368,I-3005,7.00,L,Y
706,Prepared Frz,Pro - Marinara Sauce,$19.9424,18.00,$0.3147,SPICE CHILI RED PEPPER CRUSHED,g,P-15368,I-3804,7.00,L,Y
707,Prepared Frz,Pro - Marinara Sauce,$19.9424,500.00,$2.2223,Onion White Peeled,g,P-15368,I-6865,7.00,L,Y
708,Prepared Frz,Pro - Marinara Sauce,$19.9424,1.00,$6.7740,Sawmill Creek - White 16L,L,P-15368,I-9117,7.00,L,Y


In [28]:
Products[~Products['line_item_num'].str.startswith('P-')].head()

,item_descrip,PLU,uom,item_descrip.1,item_num,inv_flag,line_item_num
18,2024S Hummus,1.00,ea,PITA POCKETS THIN,14296,Y,I-14127
19,2024S Hummus,1.00,ea,"PITA THICK 5""",14296,Y,I-14126
20,2024S Hummus,0.01,bunch,PARSLEY,14296,Y,I-2087
21,2024S Hummus,5.00,g,Olive Kalamata Superior Pitted,14296,Y,I-5731
22,2024S Hummus,15.00,ml,Olive Oil Extra virgin,14296,Y,I-14790


In [30]:
# Filter out rows where 'line_item_num' does not start with 'P-'
temp1 = Preps[~Preps['line_item_num'].str.startswith('P-')]

# Select relevant columns
temp1 = temp1[['line_item_num', 'item_descrip.1', 'line_qty', 'uom', 'pak_physical_yield', 'pak_uom', 'inv_flag']]

temp2 = Products[~Products['line_item_num'].str.startswith('P-')]

temp2 = temp2[['line_item_num', 'item_descrip.1', 'PLU', 'uom', 'inv_flag']]
temp2.rename(columns={'PLU': 'line_qty'}, inplace=True)

items = pd.concat([temp1, temp2], ignore_index=True)

# Rename columns for clarity
items.rename(columns={
    'line_item_num': 'ItemId',
    'item_descrip.1': 'Description',
    'line_qty': 'CaseQty',
    'uom': 'CaseUOM',
    'pak_physical_yield': 'PakQty',
    'pak_uom': 'PakUOM',
    'inv_flag': 'InventoryGroup'
}, inplace=True)

# Define a function to deal with commas in numeric fields
def dealWithComma(x):
    if isinstance(x, str):
        return x.replace(",", "")
    return x

# Apply the function to the 'CaseQty' column
items['CaseQty'] = items['CaseQty'].apply(dealWithComma)

# Convert 'CaseQty' to float
items = items.astype({"CaseQty": float})

# Remove duplicates based on 'ItemId'
items.drop_duplicates(subset=["ItemId"], inplace=True)

# Reset index for the final DataFrame
items.reset_index(drop=True, inplace=True)

# Display the resulting DataFrame
items


,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-1927,JALAPENO PEPPER - FRESH,20.0,g,1.25,L,Y
1,I-1958,KETCHUP VOL PAK,660.0,ml,1.25,L,Y
2,I-2025,MUSTARD DRY,22.5,g,1.25,L,Y
3,I-2323,SPICE CAYENNE SHAKER,15.0,ml,1.25,L,Y
4,I-2324,SPICE CHILI POWDER,20.0,g,1.25,L,Y
...,...,...,...,...,...,...,...
213,I-4197,DRESSING Ranch BtrMilk,2.0,oz (fl),NaN,NaN,Y
214,I-4198,Lemon Pepper Seasoning,10.0,g,NaN,NaN,Y
215,I-4199,ITEM G21- MANGO HABANERO SAUCE,2.0,oz (fl),NaN,NaN,Y
216,I-4200,Fries Sweet Potato,400.0,g,NaN,NaN,Y


In [31]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "AMS_data", "Items_List.csv")
items.to_csv(path, index = False, header = True)

### Extracting all Ingredients

In [32]:
# Ensure 'item_num' and 'line_item_num' columns are strings
Products = Products.astype({"item_num": str, "line_item_num": str})

# Prepend "R-" to 'item_num' column
Products['item_num'] = "R-" + Products['item_num']

# Display the modified DataFrame
Products

,item_descrip,PLU,uom,item_descrip.1,item_num,inv_flag,line_item_num
0,2024S Beets Salad,1.0,ea,2023 Beets Salad Prep,R-2654,Y,P-18275
2,2024S Blackend Chick Carbonara,1.0,ea,2023 Blackened Carbonara Prep,R-7227,Y,P-5506
3,2024S Blackend Chick Carbonara,2.0,Tbsp,2023 Herb oil,R-7227,Y,P-17013
5,2024S Chicken Caesar (Wrap),1.0,ea,2023 Chicken Caesar wrap Prep,R-8990,Y,P-14560
7,2024S Chicken Pesto Penne,1.0,ea,2023 Pesto Chicken,R-5554,Y,P-18295
...,...,...,...,...,...,...,...
135,2024S Wing Honey Garlic,5.0,g,GARLIC WHOLE PEELED,R-2084,Y,I-1874
136,2024S Wing Honey Garlic,20.0,g,BUTTER REG SALTED,R-2084,Y,I-2037
137,2024S Wing Honey Garlic,30.0,g,HONEY LIQUID SQUEEZE CAN,R-2084,Y,I-1917
138,2024S Wing Honey Garlic,1.0,g,SALT KOSHER COARSE,R-2084,Y,I-4091


In [33]:
Ingredients = Products[['line_item_num', 'PLU', 'uom', 'item_num']].copy()

Ingredients.rename(columns={'line_item_num': 'IngredientId', 'PLU': 'Qty', 'uom': 'Uom', 'item_num': 'Recipe'}, inplace=True)

Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)
Ingredients

,IngredientId,Qty,Uom,Recipe
0,P-18275,1.0,ea,R-2654
1,P-5506,1.0,ea,R-7227
2,P-17013,2.0,Tbsp,R-7227
3,P-14560,1.0,ea,R-8990
4,P-18295,1.0,ea,R-5554
...,...,...,...,...
100,I-1874,5.0,g,R-2084
101,I-2037,20.0,g,R-2084
102,I-1917,30.0,g,R-2084
103,I-4091,1.0,g,R-2084


In [34]:
temp = Preps[['line_item_num', 'item_num', 'line_qty', 'uom']].copy()
temp.rename(columns={'line_item_num': 'IngredientId', 'item_num': 'Recipe', 'line_qty': 'Qty', 'uom': 'Uom'}, inplace=True)
temp.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)
temp.reset_index(drop=True, inplace=True)
temp

,IngredientId,Recipe,Qty,Uom
0,I-1927,P-6068,20.00,g
1,I-1958,P-6068,660.00,ml
2,I-2025,P-6068,22.50,g
3,I-2323,P-6068,15.00,ml
4,I-2324,P-6068,20.00,g
...,...,...,...,...
695,I-3005,P-15368,60.00,g
696,I-3804,P-15368,18.00,g
697,I-6865,P-15368,500.00,g
698,I-9117,P-15368,1.00,L


In [35]:
df = [Ingredients, temp]
Ingredients = pd.concat(df)

Ingredients["Qty"] = Ingredients["Qty"].apply(dealWithComma)

In [36]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "AMS_data", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Extracting Products List

In [37]:
Products = Products[['item_num', 'item_descrip', 'inv_flag']]

Products.rename(columns={'item_num': 'ProdId', 'item_descrip': 'Description', 'inv_flag': 'SalesGroup'}, inplace=True)
Products.drop_duplicates(inplace=True)
Products.reset_index(drop=True, inplace=True)
Products

/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_70459/4244662357.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Products.rename(columns={'item_num': 'ProdId', 'item_descrip': 'Description', 'inv_flag': 'SalesGroup'}, inplace=True)
/var/folders/bq/zkgc6_d969x_x5nsl5fy0vkh0000gn/T/ipykernel_70459/4244662357.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Products.drop_duplicates(inplace=True)


,ProdId,Description,SalesGroup
0,R-2654,2024S Beets Salad,Y
1,R-7227,2024S Blackend Chick Carbonara,Y
2,R-8990,2024S Chicken Caesar (Wrap),Y
3,R-5554,2024S Chicken Pesto Penne,Y
4,R-10589,2024S Chicken Po'Boy,Y
5,R-14525,2024S Gallery Nachos,Y
6,R-8428,2024S Grilled Caesar (Salad),Y
7,R-14296,2024S Hummus,Y
8,R-6977,2024S Poutine,Y
9,R-3664,2024S Power Punch,Y


In [38]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "AMS_data", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [39]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK 23-24 Sep-Dec*", "*.oc"))
for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [40]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [41]:
# all_id_list = Items["ItemId"].unique()
all_id_set = set(Preps["item_num"].unique())
all_conv_set = set(Conversions["ConversionId"].unique())

missing_conv_id = all_id_set - all_conv_set
n = len(missing_conv_id)
print(f"{n} Items in Items dataframe but not in Conversions Dataframe:\n",)
print(missing_conv_id)

114 Items in Items dataframe but not in Conversions Dataframe:

{'P-2424', 'P-1488', 'P-17378', 'P-16793', 'P-6881', 'P-5333', 'P-15057', 'P-6068', 'P-17013', 'P-4257', 'P-9785', 'P-5506', 'P-2139', 'P-17486', 'P-4611', 'P-16778', 'P-18295', 'P-16748', 'P-10654', 'P-3615', 'P-16782', 'P-4527', 'P-1750', 'P-1106', 'P-9279', 'P-10195', 'P-18451', 'P-18268', 'P-15368', 'P-17366', 'P-7924', 'P-10674', 'P-18380', 'P-6574', 'P-16760', 'P-2535', 'P-16575', 'P-8988', 'P-16780', 'P-3286', 'P-17358', 'P-14560', 'P-15013', 'P-6514', 'P-1409', 'P-7224', 'P-6925', 'P-18458', 'P-18272', 'P-18329', 'P-5012', 'P-17360', 'P-18275', 'P-4582', 'P-10863', 'P-18052', 'P-4811', 'P-5557', 'P-5552', 'P-7912', 'P-18336', 'P-13736', 'P-16855', 'P-5741', 'P-18349', 'P-15007', 'P-9935', 'P-11125', 'P-2679', 'P-16856', 'P-11706', 'P-18381', 'P-9928', 'P-16844', 'P-16571', 'P-16860', 'P-4733', 'P-2307', 'P-17301', 'P-1711', 'P-16221', 'P-7201', 'P-3045', 'P-7873', 'P-4150', 'P-10459', 'P-8527', 'P-2291', 'P-5803', 

In [42]:
Conversions.shape

(444, 6)

In [43]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [44]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [46]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([items.shape, Preperations.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,218,7
Preps,114,5
Ingredients,805,4
Products,38,3
Conversions,444,6
